## **Gerrymandering Problem**

#### Implement a dynamic programming solution to the Gerrymandering Problem as defined in the accompanying presentation. 
#### Test code on synthetic and real data set(s) as indicated in exercises below.

---

A special thanks to
Robbie Hott,
Alexander DeLuca,
Kelly Farrell,
Samy Kebaish,
Grant Redfield,
Matthew Sachs,
Anita Taucher,

#### **Author: Harold Haugen**

---

### **Storage**
For data storage and retrieval SQLite is used.  Here, we establish a connection to the database and define a cursor to be used throughout.

In [4]:
import sqlite3 # https://docs.python.org/3/library/sqlite3.html
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats  as stats
import math
import numpy as np

## Establish a connection to our database
conn = sqlite3.connect('gerrymander.db')

## Create a cursor to execute commands through the connection
cursor = conn.cursor()


In [5]:
## When recreate is True,  drop all database tables and recreate them for an updated, clean deployment.

recreate = True

if recreate == True:

  cursor.execute("DROP TABLE IF EXISTS precinct")
  cursor.execute("DROP TABLE IF EXISTS party")
  cursor.execute("DROP VIEW IF EXISTS for_algo")
  conn.commit()

  # Quick verification to make sure everything was dropped
  cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
  cursor.fetchall()


### **Data and scripts on GitHub**
The scripts for building the database, including the data and schema, are in a github repository. urllib3 library is used to communicate over https.  

In [6]:
## SQL Scripts are in Github
## prepare to read from github
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
gitread = urllib3.PoolManager()


-----
### **Problem Statement: Gerrymandering**

Harold Haugen - We seek to determine whether gerrymandering is possible when N precincts, each containing an equal number of persons, are evenly distributed across two districts. Gerrymandering will be considered possible under the following assumptions:
- Each precinct has equal persons noted as M = 100.
- Precincts have a total count = N. 
- Each precinct has an assumed allocation of votes to the two parties (e.g., Republican or Democrat).
- When all precincts are allocated across the districts, these are evenly distributed. 
- When assessing one of two parties such as Republican or Democrat, upon full allocation of precincts evenly aross the Districts, is there a possible assignment scenario where one of the parties (e.g., Republican) has over a quarter of total votes across both districts. 
  - In other words, would D1 Republican and D2 Republican votes after full assignment to districts be True for R(D1) and R(D2) > mn/4, where again M are persons in each precinct and N is total precincts. Or in other words, over half in total. 

---
#### **Question**: 
Formally define the solution and state the recurrence used. Identify how it employs dynamic programming and clearly explain and justify. 

Solution must 1) Determine if Gerrymandering is possible and if gerrymandering is possible 2) provide the associated precinct re-assignment. Be clear and explain how.


### __Dynamic Programming Response__

The solution is to build an iterative method that loops through each precinct and in a depth-first approach builds a full precinct assignment to the lowest level to determine if the assignment allocation meets the Gerrymandering criteria of x,y > mn/4 such that S(n,n/2nxny) is True (e.g., meaning that all precincts are assigned) in each iteration.

Dynamic programming is used by creating a memoization matrix to capture each iteration of precinct assignments, then using this memo matrix to build the next N+1 precinct assignment until you get a completed D1,D2 = n/2 or D1+1, D2 = n/2 allocation. 

Output dataframe will only include those precinct assignment scenarnios that meet the required problem requirements. 

#### **Details**:
- At each assignment, the memo matrix will capture where each precinct is being assigned to and will roll these assignments forward to the next iteration.
- Upon the first iteration of P1 where a full assignment of precincts is complete, the method will check to determine if the assignment of precincts meets the Gerrymandering criteria and will break/stop if True.
- If True, a dataframe will be created that shows the number of precincts in D1=K, the voter allocations across X and Y, and the precinct labels associated with X and Y.
- If after going through each iteration of precincts, none of the assignment scenarios meets a Gerrymandering criteria, the method will return a False.
- Data Strucutre will be (J: Number of Precincts, K: Count of Precints assigned to D1, X: Count of R in D1, Y: Count of R in D2, 

<hr style="border:4px solid darkred">

### **Developed Gerrymandering Algorithm**

Provide ample comments and justify each line of code. You may wish to use or implement a sparse matrix (or something similar) to store the "memos". 

In [14]:
### The following chunk builds two methods, 'isGerrymanderPossible' and a helper function 'array_build'.

def isGerrymanderPossible(df):
    '''
    Determines if gerrymandering is possible given a dataframe that contains REP voting and Total votes for princints in two neighboring districts.
    It returns True or False, and if True, Prints out the Precinct split and voter split.  

    Provide more details about your implementation here ... 
    
    Use the isGerrymanderPossible() method to determine if Gerrymandering is possible. 
    
    Parameter input should be a Panadas Dataframe of size [N, 3]. Columns should be in order of 
    [PRECINCT, District, REP_VOTES] with integer values for District and REP_Votes.
   
    Method returns two values, value at isGerrymanderPossible[0] is a boolean True/False if Gerrymandering is possible and isGerrymanderPossible[1] 
    will return a dataframe of the Precints and District assignments that will allow for Gerrymandering. 
    '''
    
    ### Setup variables. 
    prec_array = np.array(df)
    precint_index = 0
    prec_data = np.empty([1,8],dtype=object) ## ['J','K','X','Y', 'Prec_List', 'True', D1, D2]
    
    ### Set up structure of precinct data input.
    for xx in range(0, prec_array.shape[0]):
        temp_array = array_build(999, prec_array[xx])  ## Use helper function below to reshape input data into the above structure.
        if temp_array[2] == 0 and temp_array[3] == 0:  ## If X and Y are zero in row, move on. 
            continue
        else:
            prec_data = np.vstack((prec_data, temp_array))    ## assign prec_data
    prec_data = np.delete(prec_data, (0), axis=0)     ## remove first row in prec_data
    n_length = prec_data.shape[0]
    
    ### Loop through each precinct row in prec_data by placing a single precinct into the Gerrymandering Memoization Matrix as a starting point. 
    for jj in range(0, n_length):  
        gerry_memo = np.empty([1,8],dtype=object) ## ['J','K','X','Y', 'Prec_List', 'True', D1, D2]
        gerry_memo = np.vstack((gerry_memo, prec_data[jj]))  ## Add scenario to memoization matrix. 
        gerry_memo = np.delete(gerry_memo, (0), axis=0)  ## Remove first row.
        j_val = 1
        
        for kk in range(precint_index, prec_array.shape[0]): ## Iterate again through each row in the original Precinct Data, from parameter 'df'.
            gerry_len = gerry_memo.shape[0]
            
            if prec_array[precint_index,1] == 0:  ## If District value is zero in data input, skip. 
                precint_index += 1
                continue 
            
            ### Build Precinct Data Array: structure ['J','K','X','Y', 'Prec_List', 'True', D1 Precincts, D2 Precincts] and combine with JJ precinct. 
            temp_array_d1 = array_build(1, prec_array[kk])
            temp_array_d1[6] = str(temp_array_d1[4])
            
            ### Build d2 copy to switch Republican votes position: structure ['J','K','X','Y', 'Prec_List', 'True', D1 Precincts, D2 Precincts]. 
            temp_array_d2 = np.copy(temp_array_d1) 
            temp_array_d2[1] = 0 ; temp_array_d2[3] = temp_array_d1[2] ; temp_array_d2[2] = 0  
            temp_array_d2[6] = '' ; temp_array_d2[7] = str(temp_array_d1[4]) 
            
            ## Iterate through each single precinct in the Gerrymandering Memo matrix and combined with the KK'th precinct from original prec. df. Skip duplicates. 
            for xx in range(0, gerry_len):
                temp_array_d1_copy = np.copy(temp_array_d1)  ;  temp_array_d2_copy = np.copy(temp_array_d2)  ## copies of the D1, D2 temp arrays
                prect_dupe = np.any(np.isin(gerry_memo[xx,4], temp_array_d1[4])) ## Dupes: If the prec # is in Memo Matrix, skip KK'th precinct.
              
                if prect_dupe==False: #### Create two scenarios adding precinct values from the KK'th precinct to the Memo precinct. 
                    temp_array_d1_copy[0] = gerry_memo[xx,0] + temp_array_d1_copy[0] ## total count
                    temp_array_d1_copy[1] = gerry_memo[xx,1] + temp_array_d1_copy[1] ## update K value
                    temp_array_d1_copy[2] = gerry_memo[xx,2] + temp_array_d1_copy[2] ## add X votes
                    temp_array_d1_copy[3] = gerry_memo[xx,3] + temp_array_d1_copy[3] ## add Y votes
                    temp_array_d1_copy[4] = np.append(gerry_memo[xx,4], temp_array_d1_copy[4])  ## update list of all precincts
                    temp_array_d1_copy[5] = False  ## set boolean
                    temp_array_d1_copy[6] = np.append(gerry_memo[xx,6], temp_array_d1_copy[6])  ## update list of D1 precints
                    temp_array_d1_copy[7] = gerry_memo[xx,7]

                    temp_array_d2_copy[0] = gerry_memo[xx,0] + temp_array_d2_copy[0] 
                    temp_array_d2_copy[1] = gerry_memo[xx,1] + temp_array_d2_copy[1] 
                    temp_array_d2_copy[2] = gerry_memo[xx,2] + temp_array_d2_copy[2] 
                    temp_array_d2_copy[3] = gerry_memo[xx,3] + temp_array_d2_copy[3] 
                    temp_array_d2_copy[4] = np.append(gerry_memo[xx,4], temp_array_d2_copy[4])
                    temp_array_d2_copy[5] = False
                    temp_array_d2_copy[6] = gerry_memo[xx,6]
                    temp_array_d2_copy[7] = np.append(gerry_memo[xx,7], temp_array_d2_copy[7])  
                    
                    M = 100 ; N = n_length  ## Set up values to evalute Gerrymandering
                    n_half = round(N/2) ; MN_val = (M*N)/4  ## Set up values to evalute Gerrymandering
                    j_val = len(temp_array_d2_copy[4])  ## number of precincts in Precinct List

                    ### Evaluate scenarios if Gerrymandering is True:
                    if j_val % 2 == 0 and temp_array_d1_copy[1] == n_half and temp_array_d1_copy[2] > MN_val and temp_array_d1_copy[3] > MN_val:
                        temp_array_d1_copy[5] = True
                    elif j_val % 2 == 1 and temp_array_d1_copy[1] == n_half and temp_array_d1_copy[2] > MN_val and temp_array_d1_copy[3] > MN_val:
                        temp_array_d1_copy[5] = True
                    elif j_val % 2 == 0 and temp_array_d2_copy[1] == n_half and temp_array_d2_copy[2] > MN_val and temp_array_d2_copy[3] > MN_val:
                        temp_array_d2_copy[5] = True
                    elif j_val % 2 == 1 and temp_array_d2_copy[1] == n_half and temp_array_d2_copy[2] > MN_val and temp_array_d2_copy[3] > MN_val:
                        temp_array_d2_copy[5] = True

                    gerry_memo = np.vstack((gerry_memo, temp_array_d1_copy))  ## Add scenario to memoization matrix.    
                    gerry_memo = np.vstack((gerry_memo, temp_array_d2_copy))  ## Add scenario to memoization matrix.
                
                else:
                    continue
            gerry_memo = gerry_memo[np.where((gerry_memo[:,0] == j_val))] ## Reduce Gerry_Memo matrix by keeping latest iter. rows / values for J-1 only.  
            gerry_len = gerry_memo.shape[0]  ## count rows in memo matrix
        
        if j_val == n_length and np.any(np.isin(gerry_memo[:,5], True))==True:  ## n_length is total rows of input; if total precincts equal, any boolean is True; break
            break
    ### Construct output DF
    gerry_df = pd.DataFrame(gerry_memo, columns=['J', 'K_D1', 'X_D1', 'Y_D2', 'All_Precincts', 'Gerrymandering_Possible', 'D1_Prec', 'D2_Prec'])
    gerry_df = gerry_df.query('Gerrymandering_Possible == True')  ## Only push out rows of combinations where Gerrymandering is True
    
    ## If in the final assignment of precints, any of the assignment scenarios is True for Gerrymandering, then return True/ Gerrymandering Data or False. 
    if np.any(np.isin(gerry_memo[:,5], True)):
        return True, gerry_df
    else:
        return False, False

#######################
#### Helper Method to build an array in the appropriate structure.     
def array_build(index, input_array): ### Method to build an array of values from the input precinct data.
    temp_array = np.empty([1,8],dtype=object) ## Structure is [j, k, x, y, Precincts, Test T/F, D1_Precints, D2_Precints]
    if index ==999:
        temp_array[0,0:4] = 0 ## Set first four spots as zeros.
        temp_array[0,0] = 1    ## first col. is 'j'.
        temp_array[0,4] = str(input_array[0]) ## add precint to col. 5. 
        temp_array[0,5] = False  ## Test boolean starts as False
    
        if input_array[1] == 1:
            temp_array[0,1] = 1
            temp_array[0,2] += input_array[2]
            temp_array[0,6] = str(input_array[0]) ## add precint D1         
        else:
            temp_array[0,3] += input_array[2]
            temp_array[0,7] = str(input_array[0]) ## add precint D2
    else:
        temp_array[0,0:4] = 0 ## Set first four spots as zeros.
        temp_array[0,0] = index    ## first col. is 'j'.
        temp_array[0,1] = 1
        temp_array[0,2] += input_array[2]
        temp_array[0,4] = str(input_array[0]) ## add precint to col. 5. 
        temp_array[0,5] = False
        # temp_array[0,6] = 'blank'  ;  # temp_array[0,7] = 'blank'
    # print(temp_array)
    return temp_array[0]
                

### Time complexity analysis | Algorithmic Analysis
A time complexity analysis of the algorithms in terms of the size and /or parameters is provided below. 

#### Time step function T(n):
- Steps 1 to 9: has a T(n) = 8 O(1) + 1 O(n) = O(1)
- Steps 10 to 59: has a T(n) = c10 O(n) * [ c10 to c14 O(1) + c15 O(n) * (c16 to c26 O(1) + c27 O(n) * (c28 to c59 O(1)) ] = O(n^3)
- Steps 60 to 68: has a T(n) = c60-c68 O(1) = O(1)
- Steps 101 to 120: has a T(n) = c101-c120 O(1) = O(1).

__Overall Result: <br>
Upon assessing the method structure, this would have a worse case of T(n) = O(n^3).__

<hr style="border:4px solid darkred">

## **Algorithm Test**
Run the algorithm on the example data set below. <br> 
Is gerrymandering possible? <br>
Create two other synthtetic data sets (dataframes ... like the one below): 
- one where gerrymandering is possible and
- one where gerrymandering is not possible.

Confirm your hypothesis using your implementation. 

#### __Test One__
- Create a test data input for the Gerrymandering algo. 

In [18]:
### Create test df:
precinct_data = pd.DataFrame()
precinct_data = pd.concat([ precinct_data, 
            pd.DataFrame([{"PRECINCT":"DUMMY ROW","District": 0,"REP_VOTES":0, "DEM_VOTES": 0, "Total_Votes": 0}])])
precinct_data = pd.concat([ precinct_data, 
            pd.DataFrame([{"PRECINCT":"92","District": 1,"REP_VOTES":65, "DEM_VOTES": 35, "Total_Votes": 100}])])
precinct_data = pd.concat([precinct_data, 
            pd.DataFrame([{"PRECINCT":"93","District": 1,"REP_VOTES":60, "DEM_VOTES": 40, "Total_Votes": 100}])])
precinct_data = pd.concat([precinct_data, 
            pd.DataFrame([{"PRECINCT":"94","District": 2,"REP_VOTES":45, "DEM_VOTES": 55, "Total_Votes": 100}])])
precinct_data = pd.concat([precinct_data, 
            pd.DataFrame([{"PRECINCT":"95","District": 2,"REP_VOTES":47, "DEM_VOTES": 53, "Total_Votes": 100}])])
precinct_data.reset_index(inplace = True)    
precinct_data.drop('index',axis=1,inplace=True)
precinct_data

,PRECINCT,District,REP_VOTES,DEM_VOTES,Total_Votes
0,DUMMY ROW,0,0,0,0
1,92,1,65,35,100
2,93,1,60,40,100
3,94,2,45,55,100
4,95,2,47,53,100


In [19]:
LetsRun = isGerrymanderPossible(precinct_data)

if LetsRun[0]:  ## If return index 0 = True
    print("GerryMandering is possible")
else:
    print("GerryMandering is not possible")

# \< insert code here. your output should confirm the result\>
LetsRun[1]  ## Returns Output DF or 'False'

GerryMandering is possible


,J,K_D1,X_D1,Y_D2,All_Precincts,Gerrymandering_Possible,D1_Prec,D2_Prec
5,4,2,110,107,"[92, 93, 94, 95]",True,"[92, 94]","[None, 93, 95]"
6,4,2,112,105,"[92, 93, 94, 95]",True,"[92, 95]","[None, 93, 94]"


----
#### __Synthetic Data Set 1 - Test where Gerrymandering is False__

In [127]:
precinct_synth_one = pd.DataFrame()
precinct_synth_one = pd.concat([ precinct_synth_one, 
            pd.DataFrame([{"PRECINCT":"DUMMY ROW","District": 0,"REP_VOTES":0, "DEM_VOTES": 0, "Total_Votes": 0}])])
precinct_synth_one = pd.concat([ precinct_synth_one, 
            pd.DataFrame([{"PRECINCT":"92","District": 1,"REP_VOTES":35, "DEM_VOTES": 65, "Total_Votes": 100}])])
precinct_synth_one = pd.concat([precinct_synth_one, 
            pd.DataFrame([{"PRECINCT":"93","District": 1,"REP_VOTES":60, "DEM_VOTES": 40, "Total_Votes": 100}])])
precinct_synth_one = pd.concat([precinct_synth_one, 
            pd.DataFrame([{"PRECINCT":"94","District": 2,"REP_VOTES":45, "DEM_VOTES": 55, "Total_Votes": 100}])])
precinct_synth_one = pd.concat([precinct_synth_one, 
            pd.DataFrame([{"PRECINCT":"95","District": 2,"REP_VOTES":47, "DEM_VOTES": 53, "Total_Votes": 100}])])
precinct_synth_one.reset_index(inplace = True)  ;  precinct_synth_one.drop('index',axis=1,inplace=True)
precinct_synth_one

,PRECINCT,District,REP_VOTES,DEM_VOTES,Total_Votes
0,DUMMY ROW,0,0,0,0
1,92,1,35,65,100
2,93,1,60,40,100
3,94,2,45,55,100
4,95,2,47,53,100


In [128]:
LetsRun = isGerrymanderPossible(precinct_synth_one)

if LetsRun[0]:
    print("GerryMandering is possible")
else:
    print("GerryMandering is not possible")

# \< insert code here. your output should confirm the result\>
LetsRun[1]

GerryMandering is not possible


False

---
#### __Synthetic Data Set 2 - Test where Ferrymandering is True__

In [21]:
precinct_synth_two = pd.DataFrame()
precinct_synth_two = pd.concat([ precinct_synth_two, 
            pd.DataFrame([{"PRECINCT":"DUMMY ROW","District": 0,"REP_VOTES":0, "DEM_VOTES": 0, "Total_Votes": 0}])])
precinct_synth_two = pd.concat([ precinct_synth_two, 
            pd.DataFrame([{"PRECINCT":"92","District": 1,"REP_VOTES":55, "DEM_VOTES": 45, "Total_Votes": 100}])])
precinct_synth_two = pd.concat([precinct_synth_two, 
            pd.DataFrame([{"PRECINCT":"93","District": 1,"REP_VOTES":60, "DEM_VOTES": 40, "Total_Votes": 100}])])
precinct_synth_two = pd.concat([precinct_synth_two, 
            pd.DataFrame([{"PRECINCT":"94","District": 2,"REP_VOTES":45, "DEM_VOTES": 55, "Total_Votes": 100}])])
precinct_synth_two = pd.concat([precinct_synth_two, 
            pd.DataFrame([{"PRECINCT":"95","District": 2,"REP_VOTES":49, "DEM_VOTES": 51, "Total_Votes": 100}])])
precinct_synth_two = pd.concat([precinct_synth_two, 
            pd.DataFrame([{"PRECINCT":"96","District": 1,"REP_VOTES":46, "DEM_VOTES": 54, "Total_Votes": 100}])])
precinct_synth_two = pd.concat([precinct_synth_two, 
            pd.DataFrame([{"PRECINCT":"97","District": 2,"REP_VOTES":80, "DEM_VOTES": 20, "Total_Votes": 100}])])

precinct_synth_two.reset_index(inplace = True)  ;  precinct_synth_two.drop('index',axis=1,inplace=True)
precinct_synth_two

,PRECINCT,District,REP_VOTES,DEM_VOTES,Total_Votes
0,DUMMY ROW,0,0,0,0
1,92,1,55,45,100
2,93,1,60,40,100
3,94,2,45,55,100
4,95,2,49,51,100
5,96,1,46,54,100
6,97,2,80,20,100


In [22]:
LetsRun = isGerrymanderPossible(precinct_synth_two)

if LetsRun[0]:
    print("GerryMandering is possible")
else:
    print("GerryMandering is not possible")

# \< insert code here. your output should confirm the result\>
LetsRun[1]

GerryMandering is possible


,J,K_D1,X_D1,Y_D2,All_Precincts,Gerrymandering_Possible,D1_Prec,D2_Prec
7,6,3,160,175,"[92, 93, 94, 95, 96, 97]",True,"[92, 93, 94]","[None, 95, 96, 97]"
11,6,3,164,171,"[92, 93, 94, 95, 96, 97]",True,"[92, 93, 95]","[None, 94, 96, 97]"
13,6,3,161,174,"[92, 93, 94, 95, 96, 97]",True,"[92, 93, 96]","[None, 94, 95, 97]"
22,6,3,180,155,"[92, 93, 94, 95, 96, 97]",True,"[92, 94, 97]","[None, 93, 95, 96]"
26,6,3,184,151,"[92, 93, 94, 95, 96, 97]",True,"[92, 95, 97]","[None, 93, 94, 96]"
28,6,3,181,154,"[92, 93, 94, 95, 96, 97]",True,"[92, 96, 97]","[None, 93, 94, 95]"
